In [1]:
import numpy as np
import math
np.warnings.filterwarnings('ignore')
import cv2

def debugMatrix(name, M):
    print(name, ":\n", type(M), "shape: ", M.shape, "\n", M)

def getRotationMatrixX(ax, rad=False):
    if not rad:
        ax = np.radians(ax)
    sx = np.sin(ax)
    cx = np.cos(ax)
    return np.array(((1, 0, 0), (0, cx, -sx), (0, sx, cx)))

def getRotationMatrixY(ay, rad=False):
    if not rad:
        ay = math.radians(ay)
    sy = np.sin(ay)
    cy = np.cos(ay)
    return np.array(((cy, 0, sy), (0, 1, 0), (-sy, 0, cy)))

def getRotationMatrixZ(az, rad=False):
    if not rad:
        az = math.radians(az)
    sz = np.sin(az)
    cz = np.cos(az)
    return np.array(((cz, -sz, 0), (sz, cz, 0), (0, 0, 1)))

def getRotationMatrix(ax,ay,az,rad=False,order="XYZ",debug=False):
    X = getRotationMatrixX(ax, rad)
    Y = getRotationMatrixY(ay, rad)
    Z = getRotationMatrixZ(az, rad)

    if debug:
        debugMatrix("Rotation X", X)
        debugMatrix("Rotation Y", Y)
        debugMatrix("Rotation Z", Z)

    if order == "XYZ":
        return Z @ Y @ X
    elif order == "ZYX":
        return X @ Y @ Z
    elif order == "YXZ":
        return Z @ X @ Y
    elif order == "YZX":
        return X @ Z @ Y
    elif order == "XZY":
        return Y @ Z @ X
    elif order == "ZXY":
        return Y @ X @ Z

# Return camera to world and world to camera
def getHomoTransforms(rotation_matrix, translation_matrix, debug=False):
    # The only rotation is about x
    debugMatrix("R", rotation_matrix)
    debugMatrix("T", translation_matrix)
    H_c_w = np.block([[rotation_matrix, translation_matrix], [0,0,0,1]]) 
    # Get transformation from world to camera.
    H_w_c = np.linalg.inv(H_c_w)
    if debug:
        debugMatrix("H_w_c", H_w_c)
        debugMatrix("H_c_w", H_c_w)
    return H_c_w, H_w_c

# Return Matrix K
# f: focal_length
# sx: sensor size x
# sy: sensor size y
# center: pixel location [x,y] of optical center of image
def getIntrinsicCamMatrix(f, sx, sy, center=[0,0]):
    return np.array([[f/sx, 0, center[0]], [0, f/sy, center[1]], [0, 0, 1]])

# Returns Mext
def getExtrinsicCamMatrix(H_w_c):
    Mext = H_w_c[0:3, :]
    return Mext

# For no rotation or translation
def threeDToTwoDProjection(f, sx, sy, P, cx=0, cy=0):
    return np.array((f/sx)*P[0]/P[2]+cx, (f/sy)*P[1]/P[2]+cy, 1)


def threeDToTwoDProjection(P_w,                                                                                                              rotation_matrix,
                           translation_matrix,
                           f, sx, sy,
                           cx=0,cy=0,
                           debug=False):
    if (P_w.size == 3):
        P_w = np.array([P_w,1])

    K = getIntrinsicCamMatrix(f,sx,sy,[cx,cy])
    Mext = getExtrinsicCamMatrix(getHomoTransforms(rotation_matrix,translation_matrix, debug=debug)[1])
    p = np.array(K @ Mext @ P_w)
    p /= p[2]
    p = np.hstack(p)

    if debug:
        debugMatrix("p", p)
        debugMatrix("K", K)
        debugMatrix("Mext", Mext)
        debugMatrix("P_w", P_w)

    return p



In [2]:
#Camera Angles
ax1 = 0
ay1 = 0
az1 = 30
ax2 = 30
ay2 = 0
az2 = 0
ax3 = 0
ay3 = 0
az3 = 0

# Camera Parameters
height = 480
width = 640
focal = 600
cx = width/2.0
cy = height/2.0

# Translation Matrix
translation = np.array([[6,-8,4.5]]).T

# World Points
points = np.array([[-1, -1, 0, 1],
                   [1, -1, 0, 1],
                   [1, 1, 0, 1],
                   [-1, 1, 0, 1],
                   [0,0,3,1]])


try:
    #  Give the 3x3 rotation matrix corresponding to the rotations above.
    R1 = getRotationMatrix(ax1,ay1,az1,rad=False,order="ZYX",debug=True)
    R2 = getRotationMatrix(ax2,ay2,az2,rad=False,order="XYZ",debug=True)
    R3 = getRotationMatrix(ax3,ay3,az3,rad=False,order="XYZ",debug=True)

    R = R3 @ R2 @ R1
    # Compute the homogeneous transformation matrix that represents the pose
    # of the camera with respect to the world
    # Compute the homogeneous transformation matrix that represents the pose 
    # of the world with respect to the camera
    H_c_w, H_w_c = getHomoTransforms(R, translation, debug=True)
    debugMatrix("H_c_w", H_c_w)
    debugMatrix("H_w_c", H_w_c)

    # Write the intrinsic camera calibration matrix K.
    K = getIntrinsicCamMatrix(focal, focal/width, focal/height, center=[cx,cy])
    debugMatrix("K", K)

    # Find new points on image
    Mext = H_w_c[0:3, :]
    new_points = []
    for point in points:
        new_point = K @ Mext @ point
        new_point = new_point/new_point[2]
        new_points.append(new_point)

    # Create a blank (zeros) image, and project the 7 points to the image.
    # Write white dots into the image at those points. 
    image = np.zeros((width,height, 3))
    for point in new_points:
        image[int(point[1])][int(point[0])] = (0,0,255)

    # Using OpenCV's “line” function, draw lines between the points on the image. 
    # Namely draw a line from point 0 to point 1, another line from point 1 
    # to point 2, and so forth. 
    for i in range(len(new_points) -1):
        cv2.line(image,(int(new_points[i][0]),int(new_points[i][1])), (int(new_points[i+1][0]), int(new_points[i+1][1])),(255,0,0),thickness=1)

    # Save image
    cv2.imwrite("Pyramid_Camera_View.jpg", image)
    cv2.imshow("Pyramid Camera View", image)
    cv2.waitKey(0)
finally:
    cv2.destroyAllWindows()

Rotation X :
 <class 'numpy.ndarray'> shape:  (3, 3) 
 [[ 1.  0.  0.]
 [ 0.  1. -0.]
 [ 0.  0.  1.]]
Rotation Y :
 <class 'numpy.ndarray'> shape:  (3, 3) 
 [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [-0.  0.  1.]]
Rotation Z :
 <class 'numpy.ndarray'> shape:  (3, 3) 
 [[ 0.8660254 -0.5        0.       ]
 [ 0.5        0.8660254  0.       ]
 [ 0.         0.         1.       ]]
Rotation X :
 <class 'numpy.ndarray'> shape:  (3, 3) 
 [[ 1.         0.         0.       ]
 [ 0.         0.8660254 -0.5      ]
 [ 0.         0.5        0.8660254]]
Rotation Y :
 <class 'numpy.ndarray'> shape:  (3, 3) 
 [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [-0.  0.  1.]]
Rotation Z :
 <class 'numpy.ndarray'> shape:  (3, 3) 
 [[ 1. -0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
Rotation X :
 <class 'numpy.ndarray'> shape:  (3, 3) 
 [[ 1.  0.  0.]
 [ 0.  1. -0.]
 [ 0.  0.  1.]]
Rotation Y :
 <class 'numpy.ndarray'> shape:  (3, 3) 
 [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [-0.  0.  1.]]
Rotation Z :
 <class 'numpy.ndarray'> shape:  (3, 3) 
 [[ 1. -0.  

IndexError: index 679 is out of bounds for axis 0 with size 480